# GSPE Example File
This notebook runs HOPP and applies the correct power electronics efficency factors established in the Green Steel Power Electronics (GSPE) simulator

NOTE: all of the economic analysis will use the efficency factors that are default in PySAM/set in HOPP. The goal of this script is to solely return a power time series with teh correct efficency factors.

In [1]:
from hopp.simulation import HoppInterface
import numpy as np

hi = HoppInterface("./inputs/08-GSPE.yaml")
hi.simulate(30)

print("simulation complete")

/Users/esharafu/Dev/HOPP/examples/log/hybrid_systems_2024-02-08T15.48.29.010271.log


KeyboardInterrupt: 

## Work on implementing new power electronic efficencies according to the GSPE code

1) revert efficency factors applied in HOPP

In [ ]:
with hi.hopp.system.technologies['wind'] as wp:
        DEFAULT_PYSAM_LOSSES = 12.83
        wp.system_model.gen = wp.system_model.gen * ((100 - DEFAULT_PYSAM_LOSSES)/100) ** -1 # revert PySAM default losses
        wp.system_model.annual_energy = np.sum(wp.system_model.gen)
        wp.system_model.capacity_factor = np.sum(wp.system_model.gen) / (8760 * wp.system_model.system_capacity) * 100

    else: # wp.config.model_name == 'pysam'
        pass

2) calculate losses according to GSPE code

In [6]:
eff = {'back_to_back': 0.9604,
       'diode_based_rectifier': 0.96,
       'thyristor_based_rectifier': 0.992,
       'IGBT_based_inverter': 0.98,
       'bi_DCDC_converter': 0.985,
       'uni_DCDC_converter': 0.985,
       'transformer_up': 0.99,
       'transformer_down': 0.99,
       'rectifier_power_factor': 0.89,
       'transformer_power_factor': 0.85,
       }

# TODO instead of "gutting" the code, could package the file and then use as module

DFIG_DC_1_eff = (1 - ((1 - eff['back_to_back']) / 3)) * eff['transformer_up'] * \
               eff['diode_based_rectifier'] * eff['uni_DCDC_converter'] * 100

# DFIG_DC_2_eff = (1 - ((1 - eff['back_to_back']) / 3)) * eff['transformer_up'] * \
#                eff['diode_based_rectifier'] * eff['uni_DCDC_converter'] * 100

# seems to be same efficency for both cfgs

# TODO apply the above efficency to the powers in the HI
# TODO this is done after the hopp simulaiton is complete, meaning any 
# costs assoicated with the power generation are going to have to be recalculated...
# not sure if there is a better way of doing this without completely changing HOPP

